# 要約 
このJupyter Notebookは、KerasおよびKerasNLPを使用して、LMSYS - Chatbot Arenaコンペティションにおける人間による好み予測のためのモデルを構築するスターターノートブックです。具体的には、LLM（大規模言語モデル）によって生成されたチャットボットの応答に対して、どの応答がユーザーに好まれるかを予測する問題に取り組んでいます。

## 問題
コンペティションの目的は、ユーザーが提示するプロンプトに対して複数の応答から選んだ場合に、その選ばれる可能性の高い応答を予測することです。これは、多数のLLMからの応答に基づいて行われます。

## 使用している手法
このノートブックでは、**DebertaV3**というモデルをファインチューニングし、選択肢問題の形式で応答を処理しています。具体的には、各プロンプトと応答をペアで使用し、LLMがどの応答を好ましく評価するかを学習します。モデルにはKerasNLPを使用し、混合精度トレーニングを利用することでトレーニング時間を短縮し、GPUのメモリ使用量を減らしています。

## 使用しているライブラリ
- **Keras**: モデルの構築とトレーニングに使用され、ユーザーが指定したバックエンド（TensorFlow、PyTorch、JAX）を選択して使用できます。
- **KerasNLP**: NLPのために設計されたKerasのサブライブラリで、特にDebertaV3のような様々な事前学習モデルの実装を提供しています。
- **TensorFlow**: モデルのトレーニングと評価を行うためのエコシステム。
- **NumPy/Pandas**: データ操作および分析に使用。
- **Matplotlib/Plotly**: データの可視化に使用。

## その他
ノートブックは、データの読み込みから前処理、モデル構築、訓練、予測、提出ファイルの作成までのプロセスが詳細に示されており、各ステップでのコードが含まれています。多段階のアプローチを取り入れており、最良のモデル評価を行うためのチェックポイントも設定されています。また、今後の改善ポイントとして、大規模モデルへの拡大、クロスバリデーション、応答の順序シャッフル、エポック数の増加などが提案されています。

---


# 用語概説 
以下は、Jupyter Notebookに関連する専門用語の簡単な解説です。特に、初心者がつまずきやすい詳細や実務経験がないと馴染みが薄いものに焦点を当てています。

### 専門用語の解説

1. **ファインチューニング (Fine-tuning)**:
   事前に学習したモデルに新たなデータを用いて追加の学習を行うこと。特に転移学習の文脈で、新しいタスクに特化させるために用いられる。

2. **混合精度 (Mixed Precision)**:
   トレーニング時に異なる精度のデータ型を混ぜて使用する技術。通常、計算処理の効率を高め、GPUメモリの使用量を削減するために使用される。たとえば、`float16`（半精度）と`float32`（単精度）を組み合わせることが一般的。

3. **前処理器 (Preprocessor)**:
   生データを機械学習モデルの入力として使用できる形式に変換するためのツール。具体的には、トークン化、正規化、パディング等が含まれる。

4. **トークン化 (Tokenization)**:
   生のテキストデータを意味のある単位（トークン）に分割するプロセス。これにより、テキストをモデルが理解しやすい形式に変換できる。

5. **パディング (Padding)**:
   シーケンスの長さを一定にするために、短いシーケンスに追加のトークン（通常はゼロ）を付与すること。これにより、ミニバッチ処理を効率的に行うことが可能になる。

6. **エンコーディング (Encoding)**:
   文字列データを数値データに変換するプロセス。特に、モデルがテキストを処理できる形式に変換するために重要で、UTF-8などの標準的な文字エンコーディングが使われる。

7. **層別化 (Stratification)**:
   データを分割する際に、特定の属性に基づいてその分布を保つようにすること。特に、異なるクラスのデータサンプルが均等に分割されるようにするために使用される。

8. **ロス (Loss)**:
   モデルの予測が正解とどれだけ乖離しているかを示す指標。学習中はこの値を最小化することを目的とする。`ログロス`は特に、クラス確率の予測精度を測るために用いられる。

9. **ウェイトシェアリング (Weight Sharing)**:
   複数の部分で同一の重みを共有すること。特に、同じ入力形式の異なる部分（例えば、異なる応答の処理）に同じモデルのパラメータを使用する手法。

10. **メトリック (Metric)**:
    モデルのパフォーマンスを評価するために使用される指標。精度やロスなど、モデルの性能を測るために選択される。

11. **入力層 (Input Layer)**:
    モデルにデータを供給するための層で、データの特徴を定義します。形状やデータ型が指定されることが一般的です。

12. **グローバル平均プーリング (Global Average Pooling)**:
    特徴マップ全体の平均を計算して出力する層。これにより、シーケンスや画像の情報を圧縮し、過剰適合を防ぐ手助けがされる。

これらの用語は、特に実務経験がない初学者にとっては馴染みが薄いものが多いですが、理解が深まると同時に機械学習や深層学習の実践的なスキル向上に役立ちます。

---


<center><img src="https://keras.io/img/logo-small.png" alt="Kerasのロゴ" width="100"><br/>
このスターターノートブックはKerasチームによって提供されています。</center>

# LMSYS - Chatbot Arena 人間による好み予測 [KerasNLP](https://github.com/keras-team/keras-nlp) と [Keras](https://github.com/keras-team/keras) を使用

<div align="center">
    <img src="https://i.ibb.co/wJMF5HL/lmsys.png">
</div>

このコンペティションでは、我々の目的は、LLM（大規模言語モデル）によって動かされるチャットボット同士の対戦において、どのLLMの応答がユーザーに好まれるかを予測することです。言い換えれば、このコンペティションの目標は、審査員の好みを予測し、特定のプロンプト/応答ペアが勝者として選ばれる可能性を決定することです。このノートブックでは、KerasNLPを使用して、このコンペティションのために**DebertaV3**モデルをファインチューニングするプロセスを案内します。この戦略は、選択肢問題（MCQ）モデルがトレーニングされる方法に似ています。さらに、トレーニングと推論を迅速化するために混合精度を使用します。

**ご存知でしたか**: このノートブックはバックエンドに依存しないため、TensorFlow、PyTorch、JAXのいずれのバックエンドにも対応しています。ただし、最良のパフォーマンスを達成するには `JAX` を使用することをお勧めします。KerasNLPとKerasは、好みのバックエンドを選択することを可能にしています。さらなる詳細は[Keras](https://keras.io/keras_3/)で確認してください。

**注意**: KerasNLPについてのより深い理解を得るためには、[KerasNLPガイド](https://keras.io/keras_nlp/)を参照してください。

# 📚 | ライブラリのインポート

In [ ]:
import os
# Kerasのバックエンドを"jax"に設定します。
# "tensorflow" または "torch" を使用することもできます。
os.environ["KERAS_BACKEND"] = "jax"  # または "tensorflow" または "torch"

# KerasNLPライブラリをインポートします。
import keras_nlp
# Kerasライブラリをインポートします。
import keras
# TensorFlowライブラリをインポートします。
import tensorflow as tf

# 数値計算のためのNumPyライブラリをインポートします。
import numpy as np 
# データ操作のためのPandasライブラリをインポートします。
import pandas as pd
# プログレスバーの表示のためのtqdmライブラリをインポートします。
from tqdm import tqdm
# JSON操作のためのjsonライブラリをインポートします。
import json

# グラフ描画のためのMatplotlibライブラリをインポートします。
import matplotlib.pyplot as plt
# Matplotlibの設定を行うためのmplモジュールをインポートします。
import matplotlib as mpl
# Plotlyでインタラクティブなグラフを作成するためのexpressモジュールをインポートします。
import plotly.express as px

## ライブラリのバージョン

In [ ]:
# 現在のTensorFlowのバージョンを出力します。
print("TensorFlow:", tf.__version__)
# 現在のKerasのバージョンを出力します。
print("Keras:", keras.__version__)
# 現在のKerasNLPのバージョンを出力します。
print("KerasNLP:", keras_nlp.__version__)

# ⚙️ | 設定

In [ ]:
class CFG:
    # ランダムシードを設定します。
    seed = 42  # ランダムシード
    # 使用する事前トレーニングモデルの名前を指定します。
    preset = "deberta_v3_extra_small_en" # 事前トレーニングモデルの名前
    # 入力シーケンスの長さを設定します。
    sequence_length = 512  # 入力シーケンスの長さ
    # トレーニングエポックの数を設定します。
    epochs = 3 # トレーニングエポック数
    # バッチサイズを指定します。
    batch_size = 16  # バッチサイズ
    # 学習率スケジューラのタイプを設定します。
    scheduler = 'cosine'  # 学習率スケジューラ
    # ラベルとその名前の対応を定義します。
    label2name = {0: 'winner_model_a', 1: 'winner_model_b', 2: 'winner_tie'}
    # 名前からラベルを取得する辞書を作成します。
    name2label = {v:k for k, v in label2name.items()}
    # クラスラベルのリストを作成します。
    class_labels = list(label2name.keys())
    # クラス名のリストを作成します。
    class_names = list(label2name.values())

# ♻️ | 再現性
ランダムシードの値を設定して、各実行で類似した結果を得られるようにします。

In [ ]:
# Kerasで使用するランダムシードを設定します。
keras.utils.set_random_seed(CFG.seed)

# 🧮 | 混合精度

このノートブックでは、トレーニングと推論の際にfloat32精度の代わりに混合精度を使用して、GPUメモリの使用量を削減します。これにより、より大きなバッチサイズを使用できるようになり、トレーニングと推論の時間を短縮することができます。

In [ ]:
# Kerasの全体的なポリシーを混合精度（mixed_float16）に設定します。
keras.mixed_precision.set_global_policy("mixed_float16")

# 📁 | データセットのパス

In [ ]:
# データセットのベースパスを設定します。
BASE_PATH = '/kaggle/input/lmsys-chatbot-arena'

# 📖 | メタデータ

コンペティションのデータセットは、ChatBot Arenaからのユーザーのインタラクションで構成されています。各インタラクションでは、審査員が2つの異なる大規模言語モデルに対して1つ以上のプロンプトを提示し、どのモデルがより満足のいく応答を提供したかを示します。トレーニングデータには`55,000`行が含まれており、テストセットには約`25,000`行が期待されています。

## ファイル

### `train.csv`
- `id`: 各行の一意の識別子。
- `model_[a/b]`: モデルの識別子。train.csvには存在しますが、test.csvには存在しません。
- `prompt`: 両方のモデルに与えられる入力プロンプト。
- `response_[a/b]`: モデル_[a/b]のプロンプトに対する応答。
- `winner_model_[a/b/tie]`: 審査員の選択（正解ターゲット）を示すバイナリ列。

### `test.csv`
- `id`: 各行の一意の識別子。
- `prompt`: 両方のモデルに与えられる入力プロンプト。
- `response_[a/b]`: モデル_[a/b]のプロンプトに対する応答。

> 注意: 各インタラクションには複数のプロンプトと応答がある場合がありますが、このノートブックでは**各インタラクションにつき1つのプロンプト**のみを使用します。すべてのプロンプトと応答を使用することもできます。また、データフレーム内のプロンプトと応答は文字列形式のリストとして提供されているため、`eval()`を使用してリテラルリストに変換する必要があります。

## トレーニングデータ

In [ ]:
# トレーニングデータを読み込みます。
df = pd.read_csv(f'{BASE_PATH}/train.csv') 

# サンプルデータ
# df = df.sample(frac=0.10)

# 最初のプロンプトとその関連する応答を取得します。
df["prompt"] = df.prompt.map(lambda x: eval(x)[0])
df["response_a"] = df.response_a.map(lambda x: eval(x.replace("null","''"))[0])
df["response_b"] = df.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

# ラベルの変換を行います。
df["class_name"] = df[["winner_model_a", "winner_model_b" , "winner_tie"]].idxmax(axis=1)
df["class_label"] = df.class_name.map(CFG.name2label)

# サンプルを表示します。
df.head()

## テストデータ

In [ ]:
# テストデータを読み込みます。
test_df = pd.read_csv(f'{BASE_PATH}/test.csv')

# 最初のプロンプトと応答を取得します。
test_df["prompt"] = test_df.prompt.map(lambda x: eval(x)[0])
test_df["response_a"] = test_df.response_a.map(lambda x: eval(x.replace("null","''"))[0])
test_df["response_b"] = test_df.response_b.map(lambda x: eval(x.replace("null", "''"))[0])

# サンプルを表示します。
test_df.head()

## プロンプトによる応答の文脈化

我々のアプローチでは、すべての応答に対して単一のプロンプトを使用するのではなく、各応答をプロンプトで文脈化します。つまり、各応答に対して、モデルにそれぞれの応答と組み合わせた同じセットのプロンプトを提供することになります（例: `(P + R_A)`、`(P + R_B)`など）。このアプローチは、NLPにおける選択肢問題のタスクに類似しています。

> 注意: 一部のプロンプトや応答が`utf-8`でエンコードされていない場合、データローダーを作成する際にエラーが発生することがあります。その場合、空の文字列に置き換えます。

In [ ]:
# プロンプトと選択肢に基づいてオプションを作成する関数を定義します。
def make_pairs(row):
    row["encode_fail"] = False
    try:
        # プロンプトをUTF-8でエンコードし、デコードします。
        prompt = row.prompt.encode("utf-8").decode("utf-8")
    except:
        # エンコードに失敗した場合は空の文字列にします。
        prompt = ""
        row["encode_fail"] = True

    try:
        # モデルAの応答をUTF-8でエンコードし、デコードします。
        response_a = row.response_a.encode("utf-8").decode("utf-8")
    except:
        # エンコードに失敗した場合は空の文字列にします。
        response_a = ""
        row["encode_fail"] = True

    try:
        # モデルBの応答をUTF-8でエンコードし、デコードします。
        response_b = row.response_b.encode("utf-8").decode("utf-8")
    except:
        # エンコードに失敗した場合は空の文字列にします。
        response_b = ""
        row["encode_fail"] = True
        
    # プロンプトと応答を組み合わせたオプションを作成します。
    row['options'] = [f"Prompt: {prompt}\n\nResponse: {response_a}",  # モデルAからの応答
                      f"Prompt: {prompt}\n\nResponse: {response_b}"  # モデルBからの応答
                     ]
    return row

In [ ]:
# make_pairs関数をdfの各行に適用します。
df = df.apply(make_pairs, axis=1)  
# dfの最初の2行を表示します。
display(df.head(2))  

# make_pairs関数をtest_dfの各行に適用します。
test_df = test_df.apply(make_pairs, axis=1)  
# test_dfの最初の2行を表示します。
display(test_df.head(2))

## エンコーディング失敗統計

エンコーディングの問題があるサンプル数を確認してみましょう。以下のコードから、エンコーディングに失敗したサンプルはわずか$1\%$であり、$99\%$のサンプルには問題がないことがわかります。テストデータでも同様の傾向が期待できます。このため、データのごく一部を空の文字列として扱っても、トレーニングや推論に大きな影響はないでしょう。

In [ ]:
# エンコーディング失敗の統計をカウントします。
df.encode_fail.value_counts(normalize=False)

# 🎨 | 探索的データ分析 (EDA)

## LLMの分布

In [ ]:
# モデルAとモデルBのデータを結合します。
model_df = pd.concat([df.model_a, df.model_b])
# 各モデルのカウントを取得します。
counts = model_df.value_counts().reset_index()
counts.columns = ['LLM', 'Count']

# Plotlyを使用してカスタムスタイリングの棒グラフを作成します。
fig = px.bar(counts, x='LLM', y='Count',
             title='LLMの分布',
             color='Count', color_continuous_scale='viridis')

# x軸ラベルを回転させて、読みやすくします。
fig.update_layout(xaxis_tickangle=-45)  

# グラフを表示します。
fig.show()

## 勝利の分布

In [ ]:
# 勝者のカウントを取得します。
counts = df['class_name'].value_counts().reset_index()
counts.columns = ['Winner', 'Win Count']

# トレーニングデータの勝者分布を示す棒グラフを作成します。
fig = px.bar(counts, x='Winner', y='Win Count',
             title='トレーニングデータの勝者分布',
             labels={'Winner': '勝者', 'Win Count': '勝利の数'},
             color='Winner', color_continuous_scale='viridis')

# x軸とy軸のタイトルを設定します。
fig.update_layout(xaxis_title="勝者", yaxis_title="勝利の数")

# グラフを表示します。
fig.show()

# 🔪 | データ分割

以下のコードスニペットでは、`class_label`列の層別化を使用して、既存のデータをトレーニングデータと検証データに分割します。

In [ ]:
# パッケージをインポートします。
from sklearn.model_selection import train_test_split  # パッケージをインポート

# データをトレーニングデータと検証データに分割します。
train_df, valid_df = train_test_split(df, test_size=0.2, stratify=df["class_label"])

# 🍽️ | 前処理

**何をするのか:** 前処理器は入力文字列を受け取り、それを前処理されたテンソルを含む辞書（`token_ids`, `padding_mask`）に変換します。このプロセスはトークン化から始まり、入力文字列がトークンIDのシーケンスに変換されます。

**なぜ重要なのか:** 生のテキストデータは、その高次元性のためにモデル化が難しく、複雑です。テキストをトークンのコンパクトなセットに変換することで、例えば `"The quick brown fox"` を `["the", "qu", "##ick", "br", "##own", "fox"]` に変換することで、データを単純化します。多くのモデルは、特別なトークンや追加のテンソルを使用して入力を理解します。これらのトークンは、入力を分割し、パディングを特定するなどのタスクに役立ちます。すべてのシーケンスをパディングを使って同じ長さにすることで、計算効率が向上し、その後のステップがスムーズになります。

**KerasNLP**で利用可能な前処理およびトークナイザー層にアクセスするには、以下のページを確認してください:
- [前処理](https://keras.io/api/keras_nlp/preprocessing_layers/)
- [トークナイザー](https://keras.io/api/keras_nlp/tokenizers/)

In [ ]:
# DebertaV3モデルの前処理器を設定します。
preprocessor = keras_nlp.models.DebertaV3Preprocessor.from_preset(
    preset=CFG.preset, # モデルの名前
    sequence_length=CFG.sequence_length, # 最大シーケンス長、短い場合はパディングされる
)

次に、前処理層の出力形状がどのようになるかを確認しましょう。層の出力形状は $(num\_responses, sequence\_length)$ として表されます。

In [ ]:
# 最初の行のオプションを処理します。
outs = preprocessor(df.options.iloc[0])  

# 各処理された出力の形状を表示します。
for k, v in outs.items():
    print(k, ":", v.shape)

`preprocessing_fn`関数を使用して、`dataset.map(preprocessing_fn)`メソッドを介して各テキストオプションを変換します。

In [ ]:
# テキストを前処理するための関数を定義します。
def preprocess_fn(text, label=None):
    # テキストを前処理します。
    text = preprocessor(text)  
    # ラベルが利用可能であれば、処理されたテキストとラベルを返します。
    return (text, label) if label is not None else text

# 🍚 | データローダー

以下のコードは、データ処理のために`tf.data.Dataset`を使用して堅牢なデータフローパイプラインを構築します。`tf.data`の主な特徴は、パイプラインの構築を簡素化し、コンポーネントをシーケンスで表現できる点です。

`tf.data`に関する詳細は、この[ドキュメント](https://www.tensorflow.org/guide/data)を参照してください。

In [ ]:
# テキストとラベルを使用してデータセットを構築する関数を定義します。
def build_dataset(texts, labels=None, batch_size=32,
                  cache=True, shuffle=1024):
    AUTO = tf.data.AUTOTUNE  # AUTOTUNEオプション
    # ラベルが指定されていない場合、slicesをテキストのタプルに設定します。
    slices = (texts,) if labels is None else (texts, keras.utils.to_categorical(labels, num_classes=3))  # スライスを作成
    # スライスからデータセットを作成します。
    ds = tf.data.Dataset.from_tensor_slices(slices)  
    # キャッシュが有効な場合、データセットをキャッシュします。
    ds = ds.cache() if cache else ds  
    # 前処理関数をマッピングします。
    ds = ds.map(preprocess_fn, num_parallel_calls=AUTO)  
    opt = tf.data.Options()  # データセットオプションを作成
    if shuffle: 
        # シャッフルが有効な場合、データセットをシャッフルします。
        ds = ds.shuffle(shuffle, seed=CFG.seed)  
        opt.experimental_deterministic = False
    # データセットオプションを設定します。
    ds = ds.with_options(opt)  
    # データセットをバッチ化します。
    ds = ds.batch(batch_size, drop_remainder=False)  
    # 次のバッチをプリフェッチします。
    ds = ds.prefetch(AUTO)  
    return ds  # 構築されたデータセットを返します

## トレーニング/検証データローダーの構築

In [ ]:
# トレーニングデータのテキストとラベルを抽出します。
train_texts = train_df.options.tolist()  # トレーニングテキストを抽出
train_labels = train_df.class_label.tolist()  # トレーニングラベルを抽出
# トレーニングデータローダーを構築します。
train_ds = build_dataset(train_texts, train_labels,
                         batch_size=CFG.batch_size,
                         shuffle=True)

# 検証データのテキストとラベルを抽出します。
valid_texts = valid_df.options.tolist()  # 検証テキストを抽出
valid_labels = valid_df.class_label.tolist()  # 検証ラベルを抽出
# 検証データローダーを構築します。
valid_ds = build_dataset(valid_texts, valid_labels,
                         batch_size=CFG.batch_size,
                         shuffle=False)

# ⚓ | 学習率スケジュール

学習率スケジューラーの実装は、転移学習において非常に重要です。学習率は`lr_start`から始まり、さまざまな手法を用いて徐々に`lr_min`まで減少します。手法には以下が含まれます:
- `step`: ステップ状に学習率を下げる方法で、階段のように見えます。
- `cos`: コサインカーブを利用して、学習率を徐々に減少させる方法です。
- `exp`: 学習率を指数的に減少させる方法です。

**重要性:** 適切に構築された学習率スケジュールは、効率的なモデルのトレーニングに不可欠であり、最適な収束を保証し、オーバーシュートや停滞といった問題を避けるのに役立ちます。

In [ ]:
import math

# 学習率コールバックを取得する関数を定義します。
def get_lr_callback(batch_size=8, mode='cos', epochs=10, plot=False):
    lr_start, lr_max, lr_min = 1.0e-6, 0.6e-6 * batch_size, 1e-6  # 学習率の開始、最大、最小値を設定
    lr_ramp_ep, lr_sus_ep, lr_decay = 2, 0, 0.8  # 学習率ランプのエポック数、持続エポック数、減衰率を設定

    def lrfn(epoch):  # 学習率更新関数
        if epoch < lr_ramp_ep: 
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start  # ランプアップ
        elif epoch < lr_ramp_ep + lr_sus_ep: 
            lr = lr_max  # 最大値で維持
        elif mode == 'exp': 
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min  # 指数減衰
        elif mode == 'step': 
            lr = lr_max * lr_decay**((epoch - lr_ramp_ep - lr_sus_ep) // 2)  # ステップ減衰
        elif mode == 'cos':
            decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep + 3, epoch - lr_ramp_ep - lr_sus_ep
            phase = math.pi * decay_epoch_index / decay_total_epochs  # コサイン減衰
            lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
        return lr

    if plot:  # plotがTrueの場合、学習率曲線をプロットします。
        plt.figure(figsize=(10, 5))
        plt.plot(np.arange(epochs), [lrfn(epoch) for epoch in np.arange(epochs)], marker='o')
        plt.xlabel('epoch'); plt.ylabel('lr')
        plt.title('学習率スケジューラ')
        plt.show()

    return keras.callbacks.LearningRateScheduler(lrfn, verbose=False)  # 学習率コールバックを作成する

In [ ]:
# 学習率コールバックを取得します。プロットを表示します。
lr_cb = get_lr_callback(CFG.batch_size, plot=True)

# 💾 | モデルチェックポイント

以下のコードは、トレーニング中にモデルの最良のチェックポイントを保存するコールバックを作成します。このチェックポイントは、提出時の推論に使用します。

In [ ]:
# モデルチェックポイントコールバックを取得します。
ckpt_cb = keras.callbacks.ModelCheckpoint(f'best_model.weights.h5',
                                          monitor='val_log_loss',  # 検証ロスを監視
                                          save_best_only=True,  # 最良のみを保存
                                          save_weights_only=True,  # 重みのみを保存
                                          mode='min')  # 最小値で監視

# 📏 | メトリック

このコンペティションのメトリックは**ログロス**です。このメトリックは数学的に次のように表されます。

$$
\text{Log Loss} = -\frac{1}{N} \sum_{i=1}^{N} \left( y_i \log(p_i) + (1 - y_i) \log(1 - p_i) \right)
$$

ここで、$ N $はサンプルの数、$ y_i $は真のラベル、$ p_i $はサンプルが正のクラスに属する予測確率です。

このメトリックは、分類タスクで広く使用されるカテゴリカルクロスエントロピーに似ています。したがって、ロスをゼロから実装する必要はありません。Kerasライブラリにはこのメトリックの実装が既にあるため、単にこのメトリックを使用してモデルの性能を監視します。

In [ ]:
# ログロスメトリックをKerasのCategoricalCrossentropyとして定義します。
log_loss = keras.metrics.CategoricalCrossentropy(name="log_loss")

# 🤖 | モデリング

`KerasNLP`ライブラリは、`Bert`、`Roberta`、`DebertaV3`などのさまざまなNLPモデルアーキテクチャを提供します。このノートブックでは`DebertaV3`に焦点を当てていますが、他のモデルは[KerasNLPのドキュメント](https://keras.io/api/keras_nlp/models/)で探索できます。より深い理解のためには、[入門ガイド](https://keras.io/guides/keras_nlp/getting_started/)を参照してください。

我々のアプローチでは、`keras_nlp.models.DebertaV3Classifier`を利用して各プロンプトと応答のペアを処理し、出力埋め込みを生成します。その後、これらの埋め込みを連結し、プーリング層と分類器を通してログitsを取得し、最終的な出力のために`softmax`関数を適用します。

複数の応答を扱う際には、ウェイトシェアリング戦略を使用します。つまり、モデルにはプロンプトとともに1つの応答をずつ提供し、`(P + R_A)`, `(P + R_B)`などの形で全ての応答に対して同じモデルの重みを使用します。全ての応答の埋め込みを取得した後、それらを連結し、平均プーリングを適用します。次に、`Linear/Dense`層と`Softmax`関数を分類器として最終結果を取得します。一度に全ての応答を提供すると、テキストの長さが増加し、モデルの扱いが複雑になります。分類器では`winner_model_a`、`winner_model_b`、および`draw`ケースの3つのクラスを使用することに注意してください。

下の図はこのアプローチを示しています：

<div align="center">
    <img src="https://i.postimg.cc/g0gcvy3f/Kaggle-drawio.png">
</div>

コーディングの観点からは、図に示されているように別々のモデルではなく、共有重みを持つ同一のモデルをすべての応答に使用することに注意してください。

In [ ]:
# 入力層を定義します。
inputs = {
    "token_ids": keras.Input(shape=(2, None), dtype=tf.int32, name="token_ids"),
    "padding_mask": keras.Input(shape=(2, None), dtype=tf.int32, name="padding_mask"),
}
# DebertaV3Classifierバックボーンを作成します。
backbone = keras_nlp.models.DebertaV3Backbone.from_preset(
    CFG.preset,
)

# 最初の応答: (P + R_A) の埋め込みをバックボーンを使用して計算します。
response_a = {k: v[:, 0, :] for k, v in inputs.items()}
embed_a = backbone(response_a)

# 2番目の応答: (P + R_B) の埋め込みを、同じバックボーンを使用して計算します。
response_b = {k: v[:, 1, :] for k, v in inputs.items()}
embed_b = backbone(response_b)

# 最終的な出力を計算します。
embeds = keras.layers.Concatenate(axis=-1)([embed_a, embed_b])
embeds = keras.layers.GlobalAveragePooling1D()(embeds)
outputs = keras.layers.Dense(3, activation="softmax", name="classifier")(embeds)
model = keras.Model(inputs, outputs)

# オプティマイザー、損失、メトリックとともにモデルをコンパイルします。
model.compile(
    optimizer=keras.optimizers.Adam(5e-6),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.02),
    metrics=[
        log_loss,
        keras.metrics.CategoricalAccuracy(name="accuracy"),
    ],
)

### モデルサマリー

In [ ]:
# モデルの概要を表示します。
model.summary()

### モデルのプロット

以下のモデルグラフでは、**4つ**の入力があるように見えるかもしれませんが、実際には前述の通り**2つ**の入力があります。我々の入力は2つの部分から構成されており、それぞれの応答に対するものです。しかし、各入力には`token_ids`と`padding_mask`があるため、4つの入力があるように見えますが、実際には2つの入力となります。

In [ ]:
# 現在エラーが発生しています!! [おそらくライブラリや環境の問題ですので、近いうちに修正されることを願っています]

# モデルのプロットを試みますが、エラーが発生する可能性があります。
# keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)

# 🚂 | トレーニング

In [ ]:
# モデルのトレーニングを開始します。
history = model.fit(
    train_ds,
    epochs=CFG.epochs,
    validation_data=valid_ds,
    callbacks=[lr_cb, ckpt_cb]
)

## 最良モデルの読み込み

トレーニングが完了した後、最良の結果を得るために重みを読み込み、最良のパフォーマンスを得ましょう。

In [ ]:
# 最良の重みを読み込みます。
model.load_weights('/kaggle/working/best_model.weights.h5')

# 🧪 | 予測

In [ ]:
# テストデータセットを構築します。
test_texts = test_df.options.tolist()
test_ds = build_dataset(test_texts,
                         batch_size=min(len(test_df), CFG.batch_size),
                         shuffle=False)

In [ ]:
# トレーニングされたモデルを使用してテストデータに対して予測を行います。
test_preds = model.predict(test_ds, verbose=1)

# 📬 | 提出

以下のコードは、提出ファイルの準備をします。

In [ ]:
# 提出用のデータフレームを作成します。
sub_df = test_df[["id"]].copy()
# 予測結果を追加します。
sub_df[CFG.class_names] = test_preds.tolist()
# 提出ファイルをCSV形式で保存します。
sub_df.to_csv("submission.csv", index=False)
# 最初の数行を表示します。
sub_df.head()

# 🔭 | 今後の方向性

このノートブックでは、小さなモデルと控えめなトークン長で良いスコアを達成しましたが、改善の余地はまだたくさんあります。以下の方法でさらなる向上を図ることができます：

1. `Deberta-Base`や`Deberta-Small`のような大きなモデル、あるいは`Gemma`のようなLLMを試してみる。
2. 最大トークン長を増やしてデータの損失を減らす。
3. 5フォールドクロスバリデーションとアンサンブルを使用してモデルを堅牢にし、より良いスコアを得る。
4. 応答の順序をシャッフルするなどの拡張を追加して、より堅牢なパフォーマンスを得る。
5. より多くのエポックでトレーニングする。
6. 学習率スケジューラーを調整する。

# 📌 | 参考文献

* [LLM Science Exam: KerasCore + KerasNLP [TPU]](https://www.kaggle.com/code/awsaf49/llm-science-exam-kerascore-kerasnlp-tpu)
* [AES 2.0: KerasNLP Starter](https://www.kaggle.com/code/awsaf49/aes-2-0-kerasnlp-starter)